In [11]:
from datetime import datetime, timedelta
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import matplotlib.pyplot as plt
datelist = pd.date_range(datetime.today(), periods=100).tolist()
from time import sleep
import dask.dataframe as dd
from dateutil.parser import parse
from dateutil.relativedelta import relativedelta
import os
import glob
from datetime import date

from uuid import uuid4


from shapely.geometry import Polygon, MultiPolygon, LineString, GeometryCollection
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import geopandas as gpd
import seaborn as sns

import dask.dataframe as dd
import geopandas as gpd
from detect_heatwaves import detect
from datetime import timedelta
import seaborn as sns
from datetime import date
import matplotlib.pyplot as plt

In [13]:
full_df = pd.read_csv('daily_max_THI.csv')

full_df.rename(columns={'time': 'datetime'}, inplace=True)
full_df['datetime'] = pd.to_datetime(full_df['datetime'])
full_df['day'] = pd.DatetimeIndex(full_df['datetime']).day
full_df['month'] = pd.DatetimeIndex(full_df['datetime']).month
full_df['year'] = pd.DatetimeIndex(full_df['datetime']).year
full_df = full_df[~full_df['month'].isin([4, 10])]
full_df['location'] = full_df['adm2'] + ', ' + full_df['adm1']
full_df['datetime_ord'] = [date(x,y,z).toordinal() for x,y,z in zip(full_df['year'], full_df['month'], full_df['day'])]

## Find heat waves, heat wave lengths and assign unique heatwave IDs

In [29]:
full_df['pct'] = full_df.groupby('location')['thi'].transform(lambda x: x.rank(pct=True))

In [31]:
#create dictionary containing each county's heatwave periods
heatwave_dic = {}
for adm2 in full_df['location'].unique():
    adm2_df = full_df[(full_df['location'] == adm2)]
    adm2_df.reset_index(inplace=True)
    adm2_df = adm2_df.sort_values(by='datetime')
    adm2_df.drop_duplicates(['adm2', 'datetime'], inplace=True)
    heatwave_dic[adm2] = detect(adm2_df['datetime_ord'].values, adm2_df['thi'].values, pctile=90, minDuration=2)

    
heatwave_dates = {}
consec_dates = {}
heatwave_IDs = {}
for adm2 in heatwave_dic.keys(): 
    consec_dates[adm2] = {}
    heatwave_dates[adm2] = []
    heatwave_IDs[adm2] = {}


    heatwave_starts = heatwave_dic[adm2]['date_start']
    heatwave_ends = heatwave_dic[adm2]['date_end']
    for i in range(len(heatwave_starts)):
        heatwave_code = str(uuid4())
        heatwave_days = pd.date_range(heatwave_starts[i],heatwave_ends[i],freq='d')
        length_heatwaves = len(heatwave_days)
        consec_dates[adm2][heatwave_days[0]] = length_heatwaves
        for item in heatwave_days:
            heatwave_dates[adm2].append(item)
            heatwave_IDs[adm2][item] = heatwave_code
            
            
heat_df = pd.DataFrame.from_dict(heatwave_dates, orient='index').unstack().reset_index()
heat_df.rename(columns={'level_1':'location', 0:'datetime'}, inplace=True)


heat_df.dropna(inplace=True)
heat_df['heatwave']=1

In [35]:
full_df['datetime'] = pd.to_datetime(full_df['datetime'])

In [37]:
full_df = full_df.merge(heat_df, on=['location', 'datetime'], how='left')
full_df['heatwave'] = full_df['heatwave'].replace(np.nan, 0)

In [40]:
full_df[full_df['heatwave'] == 1]['pct'].min()

0.9006535947712418

In [42]:
### sanshashi is missing
len(full_df[full_df['heatwave'] == 1]['location'].unique())

374

In [50]:
codes = pd.DataFrame.from_dict(heatwave_IDs, orient='index').unstack().reset_index()

In [52]:
codes.columns=['datetime', 'location', 'heat_ID']

In [54]:
codes.dropna(inplace=True)

In [58]:
consec = pd.DataFrame.from_dict(consec_dates, orient='index').unstack().reset_index()

In [60]:
consec.columns= ['datetime', 'location', 'Heat Wave Duration']

In [66]:
consec.dropna(inplace=True)

In [68]:
consec

,datetime,location,Heat Wave Duration
0,2019-07-21,"beijingshi, beijingshi",8.0
1,2019-07-21,"tianjinshi, tianjinshi",12.0
2,2019-07-21,"shijiazhuangshi, hebeisheng",8.0
3,2019-07-21,"tangshanshi, hebeisheng",12.0
4,2019-07-21,"qinhuangdaoshi, hebeisheng",12.0
...,...,...,...
171932,2019-06-18,"taiwansheng, taiwansheng",4.0
172306,2020-06-27,"taiwansheng, taiwansheng",4.0
172680,2021-05-15,"taiwansheng, taiwansheng",3.0
173054,2021-06-03,"taiwansheng, taiwansheng",2.0


In [70]:
full_df = full_df.merge(codes, on=['datetime', 'location'])

In [76]:
full_df = full_df.merge(consec, on=['datetime', 'location'], how='left')

In [78]:
full_2023 = full_df[full_df['year'] == 2023]

In [239]:
#changed from v2
full_2023.to_csv('THI_China_90_2023.csv')
full_df.to_csv('THI_China_90_2019_2023.csv')